# ***Actividad 3*** | Aprendizaje supervisado y no supervisado

Franco Quintanilla - A00826953

## 1. Introducción teórica

El ML, o Aprendizaje de Máquina, tiene 3 principales vertientes, entre las cuales, las dos fundamentales son:

*   Aprendizaje Supervisado: Decision Tree, Random Forest, Regresión Lineal, Regresión Logarítmica, GB, MLP.
*   Aprendizaje No Supervisado: K-Means, Gaussian Mixture, PCA, DBSCAN.

Su principal diferencia se encuentra en los mismos datos, si estos se encuentran explícitamente "etiquetados", o no.

Los supervisados nos ayudan a identificar las características fundamentales entre las variables de entrada y las de salida. A diferencia, los no supervisados no contienen dichas etiquetas, por lo cual los algoritmos tratan de identificar patrones dentro de los datos para poder clasificar y agrupar los datos por características similares.


En pyspark se encuentran disponibles la mayoría de estos mismos, en este caso, usaremos los siguientes algoritmos en específico.

*   Aprendizaje Supervisado: Random Forest
*   Aprendizaje No Supervisado: K-Means

## 2. Selección de los datos

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, isnan, when, count, count, when, isnan, col, mean, stddev, min, max, to_date, datediff, floor, approx_percentile
import matplotlib.pyplot as plt
import pandas as pd
from functools import reduce
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, ClusteringEvaluator

In [3]:
spark = SparkSession.builder \
    .appName("Activiad3") \
    .config("spark.driver.memory", "4g")      \
    .config("spark.executor.memory", "8g")    \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "2g")      \
    .getOrCreate()

In [4]:
df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/MNA/3T/Big Data/CSV/mxmortality.csv",
                    inferSchema=True, header=True)
df.show(5)

+------------+----------+------------------+------------------+--------------------+---------+----------+-----------------+-------------------+-----+---------+--------------------+----+---------+-------------------+-------------------+------------------+--------------------+--------------------+--------------------+
|decease_date|birth_date|               tod|         daylength|          gdaylength|     flux|     gflux|              lat|               long|night|gr_lismex|                desc|sexo|causa_def|                 Br|                 Bt|                Bp|                 gBr|                 gBt|                 gBp|
+------------+----------+------------------+------------------+--------------------+---------+----------+-----------------+-------------------+-----+---------+--------------------+----+---------+-------------------+-------------------+------------------+--------------------+--------------------+--------------------+
|  2011-12-27|1929-01-06|16.583333333333332|11

In [5]:
# Edad
df = df.withColumn("edad_dias", datediff(col("decease_date"), col("birth_date")))
df = df.withColumn("edad", floor(col("edad_dias") / 365.25))

In [6]:
# Separamos las principales columnas que vamos a utilizar
df = df.select(df.edad, df.night, df.gr_lismex, df.sexo, df.causa_def)
df.show(5)

+----+-----+---------+----+---------+
|edad|night|gr_lismex|sexo|causa_def|
+----+-----+---------+----+---------+
|  82|false|      028|   1|     I259|
|  72| true|      030|   1|     I619|
|  86| true|      033|   1|     J64X|
|  48| true|      010|   2|     C349|
|  70|false|      028|   2|     I219|
+----+-----+---------+----+---------+
only showing top 5 rows



In [7]:
# Diccionario con combinaciones
combinaciones = {
    "A": ((20, 39), 2),  # Mujer 20-39
    "B": ((20, 39), 1),  # Hombre 20-39
    "C": ((40, 59), 2),  # Mujer 40-59
    "D": ((40, 59), 1),  # Hombre 40-59
    "E": ((60, 120), 2), # Mujer 60+
    "F": ((60, 120), 1), # Hombre 60+
}

# Crear particiones en un diccionario de DataFrames
particiones = {}
total = df.count()

In [8]:
for clave, ((edad_min, edad_max), sexo) in combinaciones.items():
    particiones[clave] = df.filter(
        (col("edad") >= edad_min) &
        (col("edad") <= edad_max) &
        (col("sexo") == sexo)
    )
    print(f"Partición {clave} lista: Edad {edad_min}-{edad_max}, Sexo {sexo}")

Partición A lista: Edad 20-39, Sexo 2
Partición B lista: Edad 20-39, Sexo 1
Partición C lista: Edad 40-59, Sexo 2
Partición D lista: Edad 40-59, Sexo 1
Partición E lista: Edad 60-120, Sexo 2
Partición F lista: Edad 60-120, Sexo 1


In [9]:
for clave, subset in particiones.items():
    count = subset.count()
    porcentaje = round((count / total) * 100, 2)
    print(f"Partición {clave}: {count} registros ({porcentaje}%)")

Partición A: 167087 registros (2.2%)
Partición B: 472717 registros (6.24%)
Partición C: 575399 registros (7.59%)
Partición D: 961036 registros (12.68%)
Partición E: 2423358 registros (31.98%)
Partición F: 2563269 registros (33.82%)


In [10]:
# Guardamos en CSVs
for clave, subset in particiones.items():
    subset.sample(False, 0.01).limit(1000).toPandas().to_csv(f"particion_{clave}.csv", index=False)
    print(f"Partición {clave} exportada")

Partición A exportada
Partición B exportada
Partición C exportada
Partición D exportada
Partición E exportada
Partición F exportada


In [11]:
# Muestreo
tamano_muestra_total = 10000

# Probabilidades de ocurrencia
proporciones = {
    "A": 0.168,  # Mujer 20–39
    "B": 0.132,  # Hombre 20–39
    "C": 0.196,  # Mujer 40–59
    "D": 0.154,  # Hombre 40–59
    "E": 0.196,  # Mujer 60+
    "F": 0.154,  # Hombre 60+
}

# Diccionario para guardar las muestras
muestras = {}

In [12]:
for clave, df_part in particiones.items():
    fraccion = proporciones[clave]
    tamano_muestra = int(tamano_muestra_total * fraccion)
    muestras[clave] = df_part.sample(False, fraction=1.0).limit(tamano_muestra)
    print(f"Muestra {clave} generada con {tamano_muestra} registros")

Muestra A generada con 1680 registros
Muestra B generada con 1320 registros
Muestra C generada con 1960 registros
Muestra D generada con 1540 registros
Muestra E generada con 1960 registros
Muestra F generada con 1540 registros


In [13]:
# Lista con todos los DF de la muestra
lista_muestras = list(muestras.values())

# Reduce
M = reduce(DataFrame.unionByName, lista_muestras)
print(f"Tamaño total de la muestra M: {M.count()} registros")

Tamaño total de la muestra M: 10000 registros


In [14]:
# Muestra final
M.toPandas().to_csv("muestra_representativa.csv", index=False)

## 3. Preparación de los datos

In [15]:
# Cargar la muestra M
M = spark.read.csv("muestra_representativa.csv", header=True, inferSchema=True)

In [16]:
# Eliminar valores nulos
M = M.dropna()

In [17]:
# Valores atipicos con percentiles
Q1 = M.approxQuantile("edad", [0.25], 0.01)[0]
Q3 = M.approxQuantile("edad", [0.75], 0.01)[0]
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtrar valores dentro del rango aceptable
M = M.filter((M["edad"] >= lower_bound) & (M["edad"] <= upper_bound))

In [18]:
# Categorical a integer
categorical_cols = [col for col, dtype in M.dtypes if dtype == "string"]
for col in categorical_cols:
    indexer = StringIndexer(inputCol=col, outputCol=col + "_index")
    M = indexer.fit(M).transform(M)

In [19]:
# Nigth a numerico
M = M.withColumn("night_num", when(M["night"] == True, 1).when(M["night"] == False, 0))

In [20]:
# Volvemos a filtrar las variables
M = M.select(M.edad, M.night_num, M.gr_lismex_index, M.sexo, M.causa_def_index)

In [21]:
M.show(5)

+----+---------+---------------+----+---------------+
|edad|night_num|gr_lismex_index|sexo|causa_def_index|
+----+---------+---------------+----+---------------+
|  26|        1|            2.0|   2|            3.0|
|  30|        0|           25.0|   2|           90.0|
|  38|        1|            2.0|   2|          780.0|
|  34|        0|            7.0|   2|            9.0|
|  21|        1|            0.0|   2|          255.0|
+----+---------+---------------+----+---------------+
only showing top 5 rows



In [22]:
# Guardamos los datos preporcesados
M.toPandas().to_csv("muestra_preprocesada.csv", index=False)

## 4. Preparación del conjunto de entrenamiento y prueba

In [23]:
# Cargar la muestra preprocesada
M = spark.read.csv("muestra_preprocesada.csv", header=True, inferSchema=True)

In [24]:
# Vectorizar las características
assembler = VectorAssembler(inputCols=["edad", "sexo", "gr_lismex_index", "causa_def_index", "night_num"], outputCol="Attributes")
M = assembler.transform(M)

In [25]:
# Como ya s eestratifico anteriormente, entonces solo aplicamos Train Tet Random Split
train, test = M.randomSplit([0.8, 0.2], seed=1804)
print(f"Tamaño del conjunto de entrenamiento: {train.count()}")
print(f"Tamaño del conjunto de prueba: {test.count()}")

Tamaño del conjunto de entrenamiento: 8062
Tamaño del conjunto de prueba: 1938


In [26]:
# Assembler
train.show(5)
test.show(5)

+----+---------+---------------+----+---------------+--------------------+
|edad|night_num|gr_lismex_index|sexo|causa_def_index|          Attributes|
+----+---------+---------------+----+---------------+--------------------+
|  20|        0|            0.0|   1|          157.0|[20.0,1.0,0.0,157...|
|  20|        0|            1.0|   2|            0.0|[20.0,2.0,1.0,0.0...|
|  20|        0|            2.0|   1|           98.0|[20.0,1.0,2.0,98....|
|  20|        0|            3.0|   1|          265.0|[20.0,1.0,3.0,265...|
|  20|        0|            3.0|   2|           64.0|[20.0,2.0,3.0,64....|
+----+---------+---------------+----+---------------+--------------------+
only showing top 5 rows

+----+---------+---------------+----+---------------+--------------------+
|edad|night_num|gr_lismex_index|sexo|causa_def_index|          Attributes|
+----+---------+---------------+----+---------------+--------------------+
|  20|        0|            3.0|   2|           97.0|[20.0,2.0,3.0,97....|


## 5 Construcción de modelos

### Aprendizaje Supervisado

In [27]:
# Random Forest
rf = RandomForestClassifier(featuresCol="Attributes", labelCol="sexo", numTrees=2)

# Entrenar el modelo
model = rf.fit(train)

# Predicciones
predictions = model.transform(test)

# Evaluar el modelo
evaluator = MulticlassClassificationEvaluator(labelCol="sexo", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Accuracy del modelo supervisado: {accuracy:.2f}")

Accuracy del modelo supervisado: 1.00


### Aprendizaje No Supervisado

In [28]:
# K-Means
kmeans = KMeans(featuresCol="Attributes", k=2)

# Entrenar el modelo
model = kmeans.fit(M)

# Predicciones
predictions = model.transform(M)

# Evaluar el modelo
evaluator = ClusteringEvaluator(featuresCol="Attributes")
silhouette = evaluator.evaluate(predictions)

print(f"Silhouette Score del modelo K-Means: {silhouette:.2f}")

Silhouette Score del modelo K-Means: 0.91


## Conclusiones

A lo largo de esta actividad aprendimos cómo construir un proyecto de ML, desde los grandes volúmenes de datos, hasta la partición, estratificación, modelado, etc.

Como podemos observar los resultados, el modelo de random forest está sobre entrenado, esto se deberá a la partición de los datos, el mismo modelaje, etc.

El modelo no supervisado tiene datos sentro de lo correctamente establecido, esto por que, como definimos un cluster de 2, y buscmos seprar a las personas por sexo (hombre o mujer) el algoritmo logra diferenciar estas caracteristicas.